Prepare our HTTP API client

In [30]:
module HttpClient =
    open System 
    open System.Net.Http 
    open System.Net.Http.Headers
    open System.IO

    type RequestMethod = 
        | Get
        | Post 

    let client = new System.Net.Http.HttpClient()

    let buildQueryFromMap (parameters: Map<string, string>) =
        use content = new FormUrlEncodedContent(parameters)
        let stream = content.ReadAsStream()
        let reader = new StreamReader(stream)
        reader.ReadToEnd()

    let handleResponseAsync req = 
        async {
            let! response = client.SendAsync(req) |> Async.AwaitTask
            // response.EnsureSuccessStatusCode() |> ignore 

            return!
                response.Content.ReadAsStringAsync()
                |> Async.AwaitTask
        }

    let getRequestAsync (endpoint: string) (someHeaders: Map<string, string> option) (someQueryParameters: Map<string, string> option) = 

        let req = new HttpRequestMessage()
        req.Method <- HttpMethod.Get

        match someHeaders with 
        | Some headers -> 
            headers 
            |> Map.toSeq 
            |> Seq.iter (fun (k, v) -> 
                req.Headers.Add(k, v)
            )
        | None -> 
            ()

        match someQueryParameters with 
        | Some parameters -> 
            req.RequestUri <- 
                Uri(endpoint + "?" + (buildQueryFromMap parameters))
        | None -> 
            req.RequestUri <- 
                Uri(endpoint)

        handleResponseAsync req


    let postRequestAsync (endpoint: string) (someHeaders: Map<string, string> option) (someQueryParameters: Map<string, string> option) (someBody: Map<string, string> option) = 

        let req = new HttpRequestMessage()
        req.Method <- HttpMethod.Get

        match someHeaders with 
        | Some headers -> 
            headers 
            |> Map.toSeq 
            |> Seq.iter (fun (k, v) -> 
                req.Headers.Add(k, v)
            )
        | None -> 
            ()

        match someQueryParameters with 
        | Some parameters -> 
            req.RequestUri <- 
                Uri(endpoint + "?" + (buildQueryFromMap parameters))
        | None -> 
            req.RequestUri <- 
                Uri(endpoint)

        match someBody with 
        | Some body -> 
            req.Content <- new FormUrlEncodedContent(body)
        | None -> 
            ()

        handleResponseAsync req

Define our Travel module which find coordiate from place

In [31]:
module Travel = 
    let locate place = 
        let queryOptions:  Map<string, string> = 
            Map.empty
                .Add("q", $"{place}")
                .Add("format", "json")
                .Add("addressdetails", "1")
        let headerOptions: Map<string, string> = 
            Map.empty
                .Add("accept", "application/json")

        async {
            return! HttpClient.getRequestAsync "https://nominatim.openstreetmap.org/search" (Some headerOptions) (Some queryOptions)
        }
        

        
        

In [32]:
Travel.locate "丹东市, 振兴区" |> Async.RunSynchronously

<html>
<head>
<title>Access blocked</title>
</head>
<body>
<h1>Access blocked</h1>

<p>You have been blocked because you have violated the
<a href="https://operations.osmfoundation.org/policies/nominatim/">usage policy</a>
of OSM's Nominatim geocoding service. Please be aware that OSM's resources are
limited and shared between many users. The usage policy is there to ensure that
the service remains usable for everybody.</p>

<p>Please review the terms and make sure that your
software adheres to the terms. You should in particular verify that you have set a
<b>custom HTTP referrer or HTTP user agent</b> that identifies your application, and
that you are not overusing the service with massive bulk requests.</p>

<p>If you feel that this block is unjustified or remains after you have adopted
your usage, you may contact the Nominatim system administrator at
nominatim@openstreetmap.org to have this block lifted.</p>
</body>
</head>


Not sure why it has this problem while in Elixir Livebook, it doesn't have this problem.